In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("IPL Union Task") \
    .getOrCreate()


In [0]:
commentary_df = spark.read.csv("dbfs:/FileStore/shared_uploads/girish792004@gmail.com/ipl_commentary_data-1.csv", header=True, inferSchema=True)
over_df = spark.read.csv("dbfs:/FileStore/shared_uploads/girish792004@gmail.com/ipl_over_data-3.csv", header=True, inferSchema=True)


In [0]:
commentary_df.printSchema()

root
 |-- year: string (nullable = true)
 |-- series_type: string (nullable = true)
 |-- series_name: string (nullable = true)
 |-- match_no: string (nullable = true)
 |-- match_type: string (nullable = true)
 |-- match_id: integer (nullable = true)
 |-- match_venue: string (nullable = true)
 |-- match_status: string (nullable = true)
 |-- match_winning_team: string (nullable = true)
 |-- match_tie_breaker: string (nullable = true)
 |-- match_toss: string (nullable = true)
 |-- umpires: string (nullable = true)
 |-- match_referee: string (nullable = true)
 |-- third_umpires: string (nullable = true)
 |-- match_datetime: string (nullable = true)
 |-- team1_name: string (nullable = true)
 |-- team2_name: string (nullable = true)
 |-- team1_score: string (nullable = true)
 |-- team1_wickets: integer (nullable = true)
 |-- team2_score: integer (nullable = true)
 |-- team2_wickets: integer (nullable = true)
 |-- team1_captain: string (nullable = true)
 |-- team1_players: string (nullable = 

In [0]:
over_df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- series_type: string (nullable = true)
 |-- series_name: string (nullable = true)
 |-- match_no: string (nullable = true)
 |-- match_type: string (nullable = true)
 |-- match_id: integer (nullable = true)
 |-- match_venue: string (nullable = true)
 |-- match_status: string (nullable = true)
 |-- match_winning_team: string (nullable = true)
 |-- match_tie_breaker: string (nullable = true)
 |-- match_toss: string (nullable = true)
 |-- umpires: string (nullable = true)
 |-- match_referee: string (nullable = true)
 |-- third_umpires: string (nullable = true)
 |-- match_datetime: string (nullable = true)
 |-- team1_name: string (nullable = true)
 |-- team2_name: string (nullable = true)
 |-- team1_score: string (nullable = true)
 |-- team1_wickets: integer (nullable = true)
 |-- team2_score: integer (nullable = true)
 |-- team2_wickets: integer (nullable = true)
 |-- team1_captain: string (nullable = true)
 |-- team1_players: string (nullable =

In [0]:
from pyspark.sql import functions as F

commentary_cols = set(commentary_df.columns)
over_cols = set(over_df.columns)

missing_in_over = commentary_cols - over_cols
missing_in_commentary = over_cols - commentary_cols

for col in missing_in_over:
    over_df = over_df.withColumn(col, F.lit(None))

for col in missing_in_commentary:
    commentary_df = commentary_df.withColumn(col, F.lit(None))


In [0]:
common_column_order = sorted(commentary_df.columns)

commentary_df = commentary_df.select(common_column_order)
over_df = over_df.select(common_column_order)

final_union_df = commentary_df.unionByName(over_df)

final_union_df.show(5)


+--------------------+-------+--------------------+--------+---------+--------------------+------------+-----------------+--------------------+----------+--------------------+--------------------+----------------------+------------------+--------------------------+----------------------+------------------+--------------------------+--------------------------+------------------------+-----------------------+--------------------------+----------------+-------+------------+---------------+--------------------+-----------+--------------------+-------------+----------+-------------+-----------+-------------------+-------------+--------------------+-------------+----------+-------------+-----------+-------------------+-------------+--------------------+--------------------+----+
|     ball_commentary|ball_no|      match_datetime|match_id| match_no|       match_referee|match_status|match_tie_breaker|          match_toss|match_type|         match_venue|  match_winning_team|over_batsman1_curr_s

In [0]:
final_union_df.columns

Out[9]: ['ball_commentary',
 'ball_no',
 'match_datetime',
 'match_id',
 'match_no',
 'match_referee',
 'match_status',
 'match_tie_breaker',
 'match_toss',
 'match_type',
 'match_venue',
 'match_winning_team',
 'over_batsman1_curr_scr',
 'over_batsman1_name',
 'over_batsman1_played_balls',
 'over_batsman2_curr_scr',
 'over_batsman2_name',
 'over_batsman2_played_balls',
 'over_bowler_bowled_maidens',
 'over_bowler_bowled_overs',
 'over_bowler_bowled_runs',
 'over_bowler_bowled_wickets',
 'over_bowler_name',
 'over_no',
 'over_summary',
 'over_total_runs',
 'series_name',
 'series_type',
 'team1_bench',
 'team1_captain',
 'team1_name',
 'team1_players',
 'team1_score',
 'team1_support_staff',
 'team1_wickets',
 'team2_bench',
 'team2_captain',
 'team2_name',
 'team2_players',
 'team2_score',
 'team2_support_staff',
 'team2_wickets',
 'third_umpires',
 'umpires',
 'year']

In [0]:
final_union_df.select("ball_commentary", "over_summary").show(10)


+--------------------+------------+
|     ball_commentary|over_summary|
+--------------------+------------+
|Siraj to Rohit, 2...|        null|
|Siraj to Rohit, n...|        null|
|Siraj to Rohit, n...|        null|
|Siraj to Rohit, 2...|        null|
|Siraj to Rohit, n...|        null|
|Siraj to Rohit, 1...|        null|
|Jamieson to Rohit...|        null|
|Jamieson to Chris...|        null|
|Jamieson to Chris...|        null|
|Jamieson to Chris...|        null|
+--------------------+------------+
only showing top 10 rows

